In [ ]:
import importlib

In [ ]:
import Data_Prep
importlib.reload(Data_Prep)
from Data_Prep import Data_Prep

data_prep = Data_Prep()

In [ ]:
pitchers, inputs, outputs, max_input_size, dates = data_prep.Generate_IO_Pitchers("WHERE (lastProspectYear<? OR lastMLBSeason<?) AND isPitcher=?", (2025,2025,1))

In [ ]:
from sklearn.model_selection import train_test_split # type: ignore
import torch
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.25, random_state=4980)

train_lengths = torch.tensor([len(seq) for seq in x_train])
test_lengths = torch.tensor([len(seq) for seq in x_test])

x_train_padded = torch.nn.utils.rnn.pad_sequence(x_train)
x_test_padded = torch.nn.utils.rnn.pad_sequence(x_test)
y_train_padded = torch.nn.utils.rnn.pad_sequence(y_train)
y_test_padded = torch.nn.utils.rnn.pad_sequence(y_test)

In [ ]:
import Pitcher_Dataset
importlib.reload(Pitcher_Dataset)
from Pitcher_Dataset import Pitcher_Dataset

train_pitchers_dataset = Pitcher_Dataset(x_train_padded, train_lengths, y_train_padded)
test_pitchers_dataset = Pitcher_Dataset(x_test_padded, test_lengths, y_test_padded)

Train Model

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
import Pitcher_Model
importlib.reload(Pitcher_Model)
from Pitcher_Model import LSTM_Model, Classification_Loss
from torch.optim import lr_scheduler
import Model_Train
importlib.reload(Model_Train)
from Model_Train import trainAndGraph
from Constants import device

batch_size = 1000
pitching_mutators = data_prep.Generate_Pitching_Mutators(batch_size, max_input_size)

num_layers = 3
hidden_size = 30
network = LSTM_Model(x_train_padded[0].shape[1], num_layers, hidden_size, pitching_mutators)
network = network.to(device)

print("Num. Parameters:", count_parameters(network))

optimizer = torch.optim.Adam(network.parameters(), lr=0.003)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20, cooldown=5, verbose=False)
loss_function = Classification_Loss

num_epochs = 1000
training_generator = torch.utils.data.DataLoader(train_pitchers_dataset, batch_size=batch_size, shuffle=True)
testing_generator = torch.utils.data.DataLoader(test_pitchers_dataset, batch_size=batch_size, shuffle=False)

graph_y_range=(0.1,0.5)
trainAndGraph(network, training_generator, testing_generator, loss_function, optimizer, scheduler, num_epochs, logging_interval=1, early_stopping_cutoff=40, should_output=True, graph_y_range=graph_y_range)

Testing

In [ ]:
network.load_state_dict(torch.load("no_name.pt"))
network.eval()
network = network.to(device)

In [ ]:
import torch.nn.functional as F

def Check_Pitcher(id : int, name : str):
    with torch.no_grad():
        for i, pitcher in enumerate(pitchers):
            if (pitcher.mlbId == id):
                input = inputs[i].unsqueeze(0)
                
                output = outputs[i]
                l = torch.tensor([input.shape[1]])
                twar, pwar, level, pa = network(input.to(device), l.to(device))
                
                torch.set_printoptions(precision=3, sci_mode=False, linewidth=1000)
                print(name)
                print(F.softmax(twar.squeeze(0).squeeze(1), dim=1).cpu())
                #print(F.softmax(level.squeeze(0).squeeze(1), dim=1).cpu())
                print(output[0][0].item())
                #print(output[0][2].item())
                return

Get Hitter Losses

In [ ]:
#Check_Pitcher(621244, "Jose Berrios")
Check_Pitcher(592102, "Cody Allen")